# Introduction to Docker

**Learning Objectives**
  * Build and run Docker containers
  * Pull Docker images from Docker Hub and Google Container Registry
  * Push Docker images to Google Container Registry

## Overview

Docker is an open platform for developing, shipping, and running applications. With Docker, you can separate your applications from your infrastructure and treat your infrastructure like a managed application. Docker helps you ship code faster, test faster, deploy faster, and shorten the cycle between writing code and running code.

Docker does this by combining kernel containerization features with workflows and tooling that helps you manage and deploy your applications.

Docker containers can be directly used in Kubernetes, which allows them to be run in the Kubernetes Engine with ease. After learning the essentials of Docker, you will have the skillset to start developing Kubernetes and containerized applications.

## Basic Docker commands

See what docker images you have. 

In [1]:
!docker images

REPOSITORY                     TAG            IMAGE ID       CREATED          SIZE
test                           test           232484aa70da   16 minutes ago   216MB
<none>                         <none>         b0453864e62e   24 minutes ago   216MB
node                           stretch-slim   52de4a21b501   5 days ago       216MB
node                           latest         ba17ecfd099c   5 days ago       991MB
hello-world                    latest         feb5d9fea6a5   6 months ago     13.3kB
gcr.io/inverting-proxy/agent   <none>         fe507176d0e6   13 months ago    1.73GB


If this is the first time working with docker you won't have any repositories listed. 

**Note**. If you are running this in an AI Notebook, then you should see a single image `gcr.io/inverting-proxy/agent`. This is the container that is currently running the AI Notebook. 

Let's use `docker run` to pull a docker image called `hello-world` from the public registry. The docker daemon will search for the `hello-world` image, if it doesn't find the image locally, it pulls the image from a public registry called Docker Hub, creates a container from that image, and runs the container for you.

In [2]:
!docker run hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



Now when we look at our docker images we should see `hello-world` there as well.

In [3]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED         SIZE
hello-world                    latest    feb5d9fea6a5   6 months ago    13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   13 months ago   1.73GB


This is the image pulled from the Docker Hub public registry. The Image ID is in `SHA256` hash format—this field specifies the Docker image that's been provisioned. When the docker daemon can't find an image locally, it will by default search the public registry for the image. Let's run the container again:

Now, if we want to run `docker run hello-world` again, it won't have to download from the container registry.

To see all docker containers running, use `docker ps`.

In [4]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED         STATUS              PORTS     NAMES
9a68ff706bd7   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   4 minutes ago   Up About a minute             proxy-agent


There are no running containers. **Note. If you are running this in at AI Notebook, you'll see one container running.**

The `hello-world` containers you ran previously already exited. In order to see all containers, including ones that have finished executing, run docker `ps -a`:

In [15]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS                      PORTS     NAMES
249ecd072dd1   hello-world                    "/hello"                 13 minutes ago   Exited (0) 12 minutes ago             nervous_chaplygin
9a68ff706bd7   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   16 minutes ago   Up 14 minutes                         proxy-agent


This shows you the Container ID, a UUID generated by Docker to identify the container, and more metadata about the run. The container Names are also randomly generated but can be specified with docker run --name [container-name] hello-world.

## Build a Docker container

Let's build a Docker image that's based on a simple node application.

**Exercise**

Open the text file called `intro.docker` in the `dockerfiles` folder and complete the TODO there. 

Your dockerfile should have the following steps

 1. use `FROM` to inherit an official Node runtime as the parent image; e.g. node:6
 2. use `WORKDIR` to seet the working directory to /app
 3. use `ADD` to copy the current directory to the container at /app
 4. use `EXPOSE` to make the containers port 80 available to the outside world
 5. use `CMD` to run the command `node ./src/app.js`

This file instructs the Docker daemon on how to build your image.

The initial line specifies the base parent image, which in this case is the official Docker image for node version 6.
In the second, we set the working (current) directory of the container.
In the third, we add the current directory's contents (indicated by the "." ) into the container.
Then we expose the container's port so it can accept connections on that port and finally run the node command to start the application.

Check out the other [Docker command references](https://docs.docker.com/engine/reference/builder/#known-issues-run) to understand what each line does.

We're going to use this Docker container to run a simple node.js app. Have a look at `app.js`. This is a simple HTTP server that listens on port 80 and returns "Hello World."


Now let's build the image. Note again the "`.`", which means current directory so you need to run this command from within the directory that has the Dockerfile.

The `-t` is to name and tag an image with the `name:tag` syntax. The name of the image is `node-app` and the tag is `0.1`. The tag is highly recommended when building Docker images. If you don't specify a tag, the tag will default to latest and it becomes more difficult to distinguish newer images from older ones. Also notice how each line in the Dockerfile above results in intermediate container layers as the image is built.

**Exercise**

Use `docker build` to build the docker image at `dockerfiles/intro.docker`. Tag the image `node-app:0.1`. 

In [47]:
!docker build -f dockerfiles/intro.docker . -t test:test

Sending build context to Docker daemon  101.4kB
Step 1/5 : FROM node:stretch-slim
 ---> 52de4a21b501
Step 2/5 : WORKDIR /app
 ---> Using cache
 ---> 20b337a04125
Step 3/5 : COPY ./src /app
 ---> Using cache
 ---> fb6b85b9d96b
Step 4/5 : EXPOSE 4000
 ---> Using cache
 ---> 7d834255aef7
Step 5/5 : CMD [ "node", "app.js" ]
 ---> Using cache
 ---> 232484aa70da
Successfully built 232484aa70da
Successfully tagged test:test


Let's check that the image has been created correctly. 

In [15]:
!docker images

REPOSITORY                     TAG            IMAGE ID       CREATED          SIZE
test                           test           232484aa70da   20 minutes ago   216MB
<none>                         <none>         b0453864e62e   28 minutes ago   216MB
node                           stretch-slim   52de4a21b501   5 days ago       216MB
node                           latest         ba17ecfd099c   5 days ago       991MB
hello-world                    latest         feb5d9fea6a5   6 months ago     13.3kB
gcr.io/inverting-proxy/agent   <none>         fe507176d0e6   13 months ago    1.73GB


In [22]:
!docker rm 86dbee782dff

86dbee782dff


You should see a `node-app` repository that was created only seconds ago. 

Notice `node` is the base image and `node-app` is the image you built. You can't remove `node` without removing `node-app` first. The size of the image is relatively small compared to VMs. Other versions of the node image such as `node:slim` and `node:alpine` can give you even smaller images for easier portability. The topic of slimming down container sizes is further explored in Advanced Topics. You can view all versions in the official repository here.

Note, you can remove an image from your docker images using `docker rmi [repository]:[tag]`.

## Run a Docker container

Now we'll run the container based on the image you built above using the `docker run` command. The `--name` flag allows you to name the container if you like. And `-p` instructs Docker to map the host's port 4000 to the container's port 80. This allows you to reach the server at http://localhost:4000. Without port mapping, you would not be able to reach the container at localhost.

In [23]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS                      PORTS     NAMES
26c5ecb75707   test:test                      "docker-entrypoint.s…"   4 minutes ago    Exited (0) 3 minutes ago              cranky_jackson
608b4fd1eb7f   hello-world                    "/hello"                 4 minutes ago    Exited (0) 4 minutes ago              nifty_haslett
ddc3d7eb3a49   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   5 minutes ago    Up 5 minutes                          proxy-agent
44fbd244ae77   test:test                      "docker-entrypoint.s…"   20 minutes ago   Exited (0) 17 minutes ago             compassionate_newton
e0dad6d84d33   node                           "docker-entrypoint.s…"   23 minutes ago   Exited (0) 23 minutes ago             zealous_yalow
fda9f0434e8f   node                           "docker-entrypoint.s…"   25 minutes ago   Exited (0) 24 minutes ago             dreamy_archimedes
83fa54b9b750   nod

**Exercise**

Use `docker run` to run the container you just build called `node-app:0.1`. Assign the host port `4000` to port `80` and assign it the name `my-app`.

In [10]:
!docker run -d -p 4000:4000 test:test -name my-app

7eb3426538d2907ea450b3d9752850120bbda0549c9c973b1077948d115da814


In [11]:
!curl http://localhost:4000

curl: (7) Failed to connect to localhost port 4000: Connection refused


To test out the server, open a terminal window and type the following command:

```bash
curl http://localhost:4000
```

You should see the server respond with `Hello World`

The container will run as long as the initial terminal is running. If you want to stop the container, run the following command in the terminal to stop and remove the container:

```bash
docker stop my-app && docker rm my-app
```
After a few moments the container will stop. You should notice the cell above will complete execution.

#### Running the container in the background
If you want to the container to run in the background (not tied to the terminal's session), you need to specify the `-d` flag.
Now run the following command to start the container in the background

**Exercise**

Modify your command above with `-d` flag to run `my-app` in the background.

In [ ]:
%%bash
!docker run -d -p 80:80 test:test

In [ ]:
!docker stop  && docker rm my-app

Your container is now running in the background. You can check the status of your running container using `docker ps`

In [ ]:
!docker ps

Notice the container is running in the output of docker ps. You can look at the logs by executing `docker logs [container_id]`. 

In [14]:
# Note, your container id will be different
!docker logs 86dbee782dff

Server running at http://0.0.0.0:80/


You should see 
```bash
Server running at http://0.0.0.0:80/
```
If you want to follow the log's output as the container is running, use the `-f` option.

## Modify & Publish

Let's modify the application and push it to your Google Cloud Repository (gcr). After that you'll remove all local containers and images to simulate a fresh environment, and then pull and run your containers from gcr. This will demonstrate the portability of Docker containers.

### Edit `app.js`
Open the file `./src/app.js` with the text editor and replace "Hello World" with another string. Then build this new image. 

**Exercise**

After modifying the `app.js` file, use `docker build` to build a new container called `node-app:0.2` from the same docker file. 

In [25]:
!docker build -f dockerfiles/intro.docker . -t test:test

Sending build context to Docker daemon  92.67kB
Step 1/5 : FROM node:stretch-slim
 ---> 52de4a21b501
Step 2/5 : WORKDIR /app
 ---> Using cache
 ---> 20b337a04125
Step 3/5 : COPY ./src /app
 ---> 28755aaec5ee
Step 4/5 : EXPOSE 4000
 ---> Running in 00e3283c8ef3
Removing intermediate container 00e3283c8ef3
 ---> af03a9563522
Step 5/5 : CMD [ "node", "app.js" ]
 ---> Running in bc6477f89097
Removing intermediate container bc6477f89097
 ---> 853f4b2bbbff
Successfully built 853f4b2bbbff
Successfully tagged test:test


Notice in `Step 2` of the output we are using an existing cache layer. From `Step 3` and on, the layers are modified because we made a change in `app.js`.

Run another container with the new image version. Notice how we map the host's port 8000 instead of 80. We can't use host port 4000 because it's already in use. 

**Exercise**

Run this new container in the background using a different port and with the name `my-app-2`.

In [26]:
!docker run -d -p 80:80 test:test

bd408fa7ce9ba31daa400ceaaa3552fe4e82a049ea26f8465d87507cd5bde550


You can check that both container are running using `docker ps`.

In [27]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED         STATUS         PORTS                          NAMES
bd408fa7ce9b   test:test                      "docker-entrypoint.s…"   4 seconds ago   Up 2 seconds   0.0.0.0:80->80/tcp, 4000/tcp   trusting_almeida
ddc3d7eb3a49   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   6 minutes ago   Up 6 minutes                                  proxy-agent


And let's test boht containers using `curl` as before:

In [29]:
!curl http://localhost:80

Hello Wolrd! - new one


In [ ]:
!curl http://localhost:4000

Recall, to stop a container running, you can execute the following command either in a terminal or (because they are running in the background) in a cell in this notebook. 

### Publish to gcr

Now you're going to push your image to the Google Container Registry (gcr). To push images to your private registry hosted by gcr, you need to tag the images with a registry name. The format is `[hostname]/[project-id]/[image]:[tag]`.

For gcr:

  * `[hostname]`= gcr.io
  * `[project-id]`= your project's ID
  * `[image]`= your image name
  * `[tag]`= any string tag of your choice. If unspecified, it defaults to "latest".

In [30]:
import os

PROJECT_ID = "qwiklabs-gcp-01-9a9d18213c32"  # REPLACE WITH YOUR PROJECT NAME

os.environ["PROJECT_ID"] = PROJECT_ID

Let's tag `node-app:0.2`.

In [31]:
!docker images

REPOSITORY                     TAG            IMAGE ID       CREATED          SIZE
test                           test           853f4b2bbbff   56 seconds ago   216MB
<none>                         <none>         232484aa70da   22 minutes ago   216MB
<none>                         <none>         b0453864e62e   31 minutes ago   216MB
node                           stretch-slim   52de4a21b501   5 days ago       216MB
node                           latest         ba17ecfd099c   5 days ago       991MB
hello-world                    latest         feb5d9fea6a5   6 months ago     13.3kB
gcr.io/inverting-proxy/agent   <none>         fe507176d0e6   13 months ago    1.73GB


docker tag busybox gcr.io/my-project/busybox**Exercise**

Tag the `node-app:0.2` image with a new image name conforming to the naming convention `gcr.io/[project-id]/[image]:[tag]`. Keep the image and tag names the same.

In [38]:
!docker tag test:test gcr.io/qwiklabs-gcp-01-9a9d18213c32/node-app:0.2

Now when we list our docker images we should see this newly tagged repository.

In [39]:
!docker images

REPOSITORY                                     TAG            IMAGE ID       CREATED          SIZE
node-app                                       0.2            853f4b2bbbff   3 minutes ago    216MB
test                                           test           853f4b2bbbff   3 minutes ago    216MB
gcr.io/qwiklabs-gcp-01-9a9d18213c32/node-app   0.2            853f4b2bbbff   3 minutes ago    216MB
<none>                                         <none>         232484aa70da   25 minutes ago   216MB
<none>                                         <none>         b0453864e62e   33 minutes ago   216MB
node                                           stretch-slim   52de4a21b501   5 days ago       216MB
node                                           latest         ba17ecfd099c   5 days ago       991MB
hello-world                                    latest         feb5d9fea6a5   6 months ago     13.3kB
gcr.io/inverting-proxy/agent                   <none>         fe507176d0e6   13 months ago    1.73GB

Next, let's push this image to gcr.

**Exercise**

Push this new image to the gcr.

In [40]:
!docker push gcr.io/qwiklabs-gcp-01-9a9d18213c32/node-app:0.2

The push refers to repository [gcr.io/qwiklabs-gcp-01-9a9d18213c32/node-app]

4f2199b5: Preparing 
2ce66471: Preparing 
25f798a2: Preparing 
b3c2e315: Preparing 
6f306d7d: Preparing 
d82b89f1: Preparing 
6f306d7d: Pushed   155.6MB/151.3MB0.2: digest: sha256:b5bab641a2c268e4cfb8b8364d50dc62f80cc1634d5a763ea86f7a9b62dd4862 size: 1781


Check that the image exists in `gcr` by visiting the image registry Cloud Console. You can navigate via the console to `Navigation menu > Container Registry` or visit the url from the cell below:

In [41]:
%%bash
echo "http://gcr.io/${PROJECT_ID}/node-app"

http://gcr.io/qwiklabs-gcp-01-9a9d18213c32/node-app


### Test the published gcr image

Let's test this image. You could start a new VM, ssh into that VM, and install gcloud. For simplicity, we'll just remove all containers and images to simulate a fresh environment.

First, stop and remove all containers using `docker stop` and `docker rm`. **Be careful not to stop the container running this AI Notebook!**.

In [42]:
!docker stop my-app && docker rm my-app

Error response from daemon: No such container: my-app


In [ ]:
!docker stop my-app-2 && docker rm my-app-2

Now remove the docker images you've created above using `docker rmi`.

In [43]:
!docker images

REPOSITORY                                     TAG            IMAGE ID       CREATED          SIZE
test                                           test           853f4b2bbbff   5 minutes ago    216MB
gcr.io/qwiklabs-gcp-01-9a9d18213c32/node-app   0.2            853f4b2bbbff   5 minutes ago    216MB
node-app                                       0.2            853f4b2bbbff   5 minutes ago    216MB
<none>                                         <none>         232484aa70da   26 minutes ago   216MB
<none>                                         <none>         b0453864e62e   35 minutes ago   216MB
node                                           stretch-slim   52de4a21b501   5 days ago       216MB
node                                           latest         ba17ecfd099c   5 days ago       991MB
hello-world                                    latest         feb5d9fea6a5   6 months ago     13.3kB
gcr.io/inverting-proxy/agent                   <none>         fe507176d0e6   13 months ago    1.73GB

In [44]:
%%bash
docker rmi node-app:0.2
docker rmi gcr.io/${PROJECT_ID}/node-app:0.2
docker rmi node-app:0.1
docker rmi node:6 
docker rmi -f hello-world:latest

Untagged: node-app:0.2
Untagged: gcr.io/qwiklabs-gcp-01-9a9d18213c32/node-app:0.2
Untagged: gcr.io/qwiklabs-gcp-01-9a9d18213c32/node-app@sha256:b5bab641a2c268e4cfb8b8364d50dc62f80cc1634d5a763ea86f7a9b62dd4862
Untagged: hello-world:latest
Untagged: hello-world@sha256:bfea6278a0a267fad2634554f4f0c6f31981eea41c553fdf5a83e95a41d40c38
Deleted: sha256:feb5d9fea6a5e9606aa995e879d862b825965ba48de054caab5ef356dc6b3412


Error: No such image: node-app:0.1
Error: No such image: node:6


Confirm all images are removed with `docker images`.

In [45]:
!docker images

REPOSITORY                     TAG            IMAGE ID       CREATED          SIZE
test                           test           853f4b2bbbff   5 minutes ago    216MB
<none>                         <none>         232484aa70da   27 minutes ago   216MB
<none>                         <none>         b0453864e62e   35 minutes ago   216MB
node                           stretch-slim   52de4a21b501   5 days ago       216MB
node                           latest         ba17ecfd099c   5 days ago       991MB
gcr.io/inverting-proxy/agent   <none>         fe507176d0e6   13 months ago    1.73GB


At this point you should have a pseudo-fresh environment. Now, pull the image and run it.

In [46]:
%%bash
docker pull gcr.io/${PROJECT_ID}/node-app:0.2
docker run -p 4000:80 -d gcr.io/${PROJECT_ID}/node-app:0.2

0.2: Pulling from qwiklabs-gcp-01-9a9d18213c32/node-app
Digest: sha256:b5bab641a2c268e4cfb8b8364d50dc62f80cc1634d5a763ea86f7a9b62dd4862
Status: Downloaded newer image for gcr.io/qwiklabs-gcp-01-9a9d18213c32/node-app:0.2
gcr.io/qwiklabs-gcp-01-9a9d18213c32/node-app:0.2
1abfbfcae85773b931b803089fc996c86701384ee1434f9edaeb4643f4d14e5f


You can check that it's running as expected using before:

In [47]:
!curl http://localhost:4000

Hello Wolrd! - new one


Copyright 2020 Google LLC Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.